In [ ]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv

from sys import stdout

from network import HandModel
from preprocess_data import SynthDataLoader, MultiBootStrapDataLoader
from utils.visualize_data import visualize_combined_map
from utils.model_utils import get_points_from_map

In [ ]:
model = HandModel()
dummy_input = tf.random.normal([1, 368, 368, 3])
_ = model(dummy_input)

loss_graph = []

model.load_weights('good progress models/vgg_19_multibootstap_epoch25/curr_vgg19_train_epoch_25.h5')

In [ ]:
copy_model = tf.keras.applications.VGG19(False, input_shape=(368, 368, 3), pooling='avg')
for i in range(1, 16):
    print(copy_model.layers[i].name)
    print(model.feature_extraction.layers[i - 1].name)
    print(model.feature_extraction.layers[i - 1].get_weights())
    model.feature_extraction.layers[i - 1].set_weights(copy_model.layers[i].get_weights())
    print(model.feature_extraction.layers[i - 1].get_weights())

In [ ]:
loader = SynthDataLoader("data/hand_labels_synth/synth2/", 32, 0.8)
train_data, val_data = loader.get_data_set()
num_batches = loader.get_num_batches()

In [ ]:
loader = MultiBootStrapDataLoader("data/hand143_panopticdb/", 16, 0.8)
train_data, val_data = loader.get_data_set()
num_batches = loader.get_num_batches()

In [ ]:
num_epochs = 25
lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=0.0001,
    decay_steps=(loader.get_num_batches() * num_epochs),  # total_epochs × steps_per_epoch
    alpha=0.0
)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule, epsilon=1e-8)

train_loss = tf.keras.metrics.Mean(name='train_loss')

In [ ]:
def compute_loss(y_pred, intermediate_pred, y):
    loss_object = tf.keras.losses.MeanSquaredError()
    final_loss = loss_object(y, y_pred)
    total_loss = final_loss
    for i in range(len(intermediate_pred)):
        total_loss += loss_object(y, intermediate_pred[i])
    
    return final_loss, total_loss

In [ ]:
def l2_loss(y_pred, intermediate_pred, y, punisment = 0.001):

    batch_size = tf.cast(tf.shape(y)[0], tf.float32)
    

    total_loss = 0
    for y_hat in intermediate_pred:
        total_loss += tf.nn.l2_loss(y - y_hat)

    final_loss = tf.nn.l2_loss(y - y_pred)

    total_loss += final_loss
    
    total_loss /= (len(intermediate_pred) + 1)

    total_loss /= batch_size
    final_loss /= batch_size

    return final_loss, total_loss

In [ ]:
def l2_punishment_loss(y_pred, intermediate_pred, y, punisment = 0.001):

    batch_size = tf.cast(tf.shape(y)[0], tf.float32)
    

    total_loss = 0
    pos_val = 0
    for y_hat in intermediate_pred:
        total_loss += tf.nn.l2_loss(y - y_hat)
        pos_val += tf.reduce_sum(y_hat[:, :, :, :-1])

    final_loss = tf.nn.l2_loss(y - y_pred)
    pos_val += tf.reduce_sum(y_pred[:, :, :, :-1])

    total_loss += final_loss
    total_loss += pos_val * punisment

    total_loss /= batch_size
    final_loss /= batch_size

    return final_loss, total_loss

In [ ]:
@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        y_pred, y_intermediate = model(x)
        fin_loss, total_loss = compute_loss(y_pred, y_intermediate, y)

    gradients = tape.gradient(total_loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return fin_loss, total_loss

In [ ]:
old_loss = 0
for epoch in range(num_epochs):
    train_loss.reset_state()


    for batch_num, (x_batch, y_batch) in enumerate(train_data):
        fin_loss, total_loss = train_step(x_batch, y_batch)
        train_loss(total_loss)
        print(f"\rProgress: {batch_num + 1}/{num_batches}", end='', flush=True)

    print()

    curr_loss = train_loss.result()

    print(f"Epoch {epoch + 1} ----- Loss: {curr_loss}")
    loss_graph.append(curr_loss)

    if curr_loss < old_loss:
        model.save(f"curr_vgg19_train_epoch_{epoch + 1}.h5")

    old_loss = curr_loss

    if curr_loss < 0.0001:
        print("Finished training, ended early")
        break


In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(loss_graph, label='Training Loss', color='blue', marker='o')

# Add labels and title
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Over Time')
plt.legend()
plt.grid(True)

# Show the plot
plt.show()

In [ ]:
np.save("good progress models/vgg_19_multibootstap_epoch25/loss_data", np.array(loss_graph))

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(15,15))

test_x, test_y, test_handpos = loader.get_random_sample()

test_x = np.expand_dims(test_x, axis = 0)


y_pred, _ = model(test_x)

# print(test_handpos)

test_handpos = test_handpos.T
axes[0].imshow(test_x[0,:,:,::-1])
# axes[0].scatter(test_handpos[0][0], test_handpos[1][0], color='green', s=50)
# axes[0].scatter(test_handpos[0][1:5], test_handpos[1][1:5], color='red', s=50)
# axes[0].scatter(test_handpos[0][5:9], test_handpos[1][5:9], color='blue', s=50)
# axes[0].scatter(test_handpos[0][9:13], test_handpos[1][9:13], color='red', s=50)
# axes[0].scatter(test_handpos[0][13:17], test_handpos[1][13:17], color='blue', s=50)
# axes[0].scatter(test_handpos[0][17:21], test_handpos[1][17:21], color='red', s=50)
# test_handpos = test_handpos.T

visualize_combined_map(test_x[0,:,:,::-1], test_y, axes[1])
visualize_combined_map(test_x[0,:,:,::-1], y_pred[0, :, :, :-1], axes[2])

pred_handpose = get_points_from_map(y_pred[0].numpy(), test_x.shape[1:3])
pred_handpose = pred_handpose.T
axes[3].imshow(test_x[0,:,:,::-1])
axes[3].scatter(pred_handpose[0][0], pred_handpose[1][0], color='green', s=50)
axes[3].scatter(pred_handpose[0][1:5], pred_handpose[1][1:5], color='red', s=50)
axes[3].scatter(pred_handpose[0][5:9], pred_handpose[1][5:9], color='blue', s=50)
axes[3].scatter(pred_handpose[0][9:13], pred_handpose[1][9:13], color='red', s=50)
axes[3].scatter(pred_handpose[0][13:17], pred_handpose[1][13:17], color='blue', s=50)
axes[3].scatter(pred_handpose[0][17:21], pred_handpose[1][17:21], color='red', s=50)
pred_handpose = pred_handpose.T


plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15,15))

test_x = cv.imread("Photo from 2025-07-24 03-05-22.552013.jpeg")

# temp = cv.blur(test_x,(10, 10))
test_x = cv.resize(test_x, (368, 368), interpolation=cv.INTER_LINEAR)

# Initialize output image
# temp = cv.resize(temp, (368, 368), interpolation=cv.INTER_CUBIC)
# temp = cv.blur(temp,(3, 3))

test_x = np.expand_dims(test_x, axis = 0)


y_pred, _ = model(test_x)

# print(test_handpos)

# test_handpos = test_handpos.T
axes[0].imshow(test_x[0,:,:,::-1])
# axes[0].scatter(test_handpos[0][0], test_handpos[1][0], color='green', s=50)
# axes[0].scatter(test_handpos[0][1:5], test_handpos[1][1:5], color='red', s=50)
# axes[0].scatter(test_handpos[0][5:9], test_handpos[1][5:9], color='blue', s=50)
# axes[0].scatter(test_handpos[0][9:13], test_handpos[1][9:13], color='red', s=50)
# axes[0].scatter(test_handpos[0][13:17], test_handpos[1][13:17], color='blue', s=50)
# axes[0].scatter(test_handpos[0][17:21], test_handpos[1][17:21], color='red', s=50)
# test_handpos = test_handpos.T

# visualize_combined_map(test_x[0,:,:,::-1], test_y, axes[1])
visualize_combined_map(test_x[0,:,:,::-1], y_pred[0, :, :, :-1], axes[1])

pred_handpose = get_points_from_map(y_pred[0].numpy(), test_x.shape[1:3])
pred_handpose = pred_handpose.T
axes[2].imshow(test_x[0,:,:,::-1])
axes[2].scatter(pred_handpose[0][0], pred_handpose[1][0], color='green', s=50)
axes[2].scatter(pred_handpose[0][1:5], pred_handpose[1][1:5], color='red', s=50)
axes[2].scatter(pred_handpose[0][5:9], pred_handpose[1][5:9], color='blue', s=50)
axes[2].scatter(pred_handpose[0][9:13], pred_handpose[1][9:13], color='red', s=50)
axes[2].scatter(pred_handpose[0][13:17], pred_handpose[1][13:17], color='blue', s=50)
axes[2].scatter(pred_handpose[0][17:21], pred_handpose[1][17:21], color='red', s=50)
pred_handpose = pred_handpose.T


plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(3, 7, figsize=(15,15))

for i, ax in enumerate(list(axes.flat)):
    ax.imshow(y_pred[0, :, :, i], cmap='gray')
    
    ax.axis('off')


plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15,15))

img_path = "test_photo.jpeg"

test_x = cv.imread(img_path)
# test_x = normalize_data(test_x)

test_x = np.expand_dims(test_x, axis = 0)

y_pred, _ = model(test_x)

IMG = test_x[0, :, :, ::-1]

axes[0].imshow(IMG)

temp = cv.resize(IMG, (80, 80), interpolation=cv.INTER_LINEAR)

# Initialize output image
output = cv.resize(temp, (368, 368), interpolation=cv.INTER_CUBIC)

axes[1].imshow(output)


plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15,15))

img_path = "test_photo.jpeg"

test_x = cv.imread(img_path)
# test_x = normalize_data(test_x)

test_x = test_x[:, ::-1, :]


temp = cv.resize(test_x, (120, 120), interpolation=cv.INTER_LINEAR)

# Initialize output image
temp = cv.resize(temp, (368, 368), interpolation=cv.INTER_CUBIC)

temp = np.expand_dims(temp, axis = 0)

y_pred, _ = model(temp)

axes[0].imshow(test_x[:, :, ::-1])

visualize_combined_map(temp[0, :, :, ::-1], y_pred[0, :, :, :-1], axes[1])


plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(3, 7, figsize=(15,15))

for i, ax in enumerate(list(axes.flat)):
    ax.imshow(y_pred[0, :, :, i], cmap='gray')
    
    ax.axis('off')


plt.tight_layout()
plt.show()